In [4]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from relife import Weibull
pd.options.plotting.backend = "plotly"

In [120]:
class Dette:
    def __init__(self,montant,annee_debut,annee_fin):
        self.montant = montant
        self.annee_debut = annee_debut
        self.anne_fin = annee_fin
        self.duree = annee_fin - annee_debut
        self.remboursement_annuel = montant/(annee_fin - annee_debut)

class Capex:
    def __init__(self,montant,annee_debut,annee_fin,duree_amortissement):
        self.montant = montant
        self.annee_debut = annee_debut
        self.anne_fin = annee_fin
        self.duree = annee_fin - annee_debut
        self.cout_annuel = montant/(annee_fin - annee_debut)
        self.duree_amortissement = duree_amortissement

class Opex:
    def __init__(self,montant,annee_debut,annee_fin):
        self.montant = montant
        self.annee_debut = annee_debut
        self.anne_fin = annee_fin
        self.duree = annee_fin - annee_debut

class Prix_MWh:
    def __init__(self,prix_MWh,annee_debut,annee_fin):
        self.prix_MWh = prix_MWh
        self.annee_debut = annee_debut
        self.anne_fin = annee_fin

class Production:
    def __init__(self,Nb_heures,capacite,annee_debut,annee_fin):
        self.Nb_heures = Nb_heures
        self.capacite = capacite
        self.production = capacite * Nb_heures
        self.annee_debut = annee_debut
        self.anne_fin = annee_fin

In [121]:
annee_debut_projet = -2
annee_fin_projet = 30

list_capex = [Capex(-1200000,-2,0,10),Capex(-1200000/12,15,17,5)]
list_opex = [Opex(-1200000*3/100,1,30)]
liste_dette = [Dette(1200000/3,-2,10), Dette(1200000/12,5,15)]
list_prix_MWh = [Prix_MWh(75,1,15),Prix_MWh(55,16,30)]
list_prod = [Production(2200,1,1,30)]

taux_actualisation = 5
taux_interets = 2
capacite = 1
prod_annuelle_MWh = 2200
Cfd_price = 75
taux_imposition = 30/100

In [124]:
df = pd.DataFrame(columns=[k for k in range(annee_debut_projet,annee_fin_projet+1)])
df.loc["Coef. actualisation"] = [(1+taux_actualisation/100)**k for k in df.columns]

df.loc["CAPEX"] = 0
for capex in list_capex:
    df.loc["CAPEX",capex.annee_debut:capex.anne_fin] += capex.cout_annuel

df.loc["OPEX"] = 0
for opex in list_opex:
    df.loc["OPEX",opex.annee_debut:opex.anne_fin] += opex.montant

df.loc["Prod. (MWh)"] = 0
for prod in list_prod:
    df.loc["Prod. (MWh)",prod.annee_debut:prod.anne_fin] = prod.production

df.loc["Revenus"] = 0
for prix_MWh in list_prix_MWh:
    df.loc["Revenus",prix_MWh.annee_debut:prix_MWh.anne_fin] += prix_MWh.prix_MWh * df.loc["Prod. (MWh)",prix_MWh.annee_debut:prix_MWh.anne_fin]

df.loc["EBITDA"] = df.loc[["OPEX","Revenus"]].sum()

df.loc["Amortissements"] = 0
for capex in list_capex:
    df.loc["Amortissements",capex.anne_fin+1:capex.anne_fin+capex.duree_amortissement] += capex.montant/capex.duree_amortissement
df.loc["EBIT"] = df.loc[["EBITDA","Amortissements"]].sum()

df.loc["Dette BOP"] = 0
df.loc["Tirage"]    = 0
df.loc["Dette EOP"] = 0
df.loc["Remboursement"] = 0

for dette in liste_dette:
    df.loc["Dette BOP",dette.annee_debut:dette.anne_fin-1] += [dette.remboursement_annuel * (dette.duree - k) for k in range(dette.duree)]
    df.loc["Tirage",dette.annee_debut] += dette.montant
    df.loc["Remboursement",dette.annee_debut:dette.anne_fin-1] += - dette.remboursement_annuel
    df.loc["Dette EOP"] = df.loc[["Dette BOP", "Remboursement"]].sum()
    df.loc["Intérêts financiers"] = - df.loc["Dette BOP"] * 2 / 100

df.loc["Résultat avant IS"] = df.loc[["EBIT","Intérêts financiers","Remboursement"]].sum()
df.loc["Impôts (IS)"] = - df.loc["Résultat avant IS"].apply(lambda x : 0 if x < 0 else x) * taux_imposition
df.loc["Résultat net"] = df.loc[["Résultat avant IS","Impôts (IS)"]].sum()
df.loc["Cash Flow"] = df.loc[["OPEX","CAPEX","Revenus","Intérêts financiers", "Impôts (IS)","Tirage","Remboursement"]].sum()
df.loc["DSCR"] = -df.loc[["EBITDA","Impôts (IS)"]].sum()/df.loc[["Intérêts financiers","Remboursement"]].sum()

fig = df.transpose().plot(kind = "bar", y = ["CAPEX","OPEX","Revenus","Impôts (IS)"]).update_layout(hovermode='x')
fig.add_trace(
    go.Scatter(
        x = df.columns,
        y = df.loc["Cash Flow"],
        name = "Cash Flow",
        marker_color = "black"
    )
)
fig.add_trace(
    go.Scatter(
        x = df.columns,
        y = df.loc["Dette BOP"],
        name = "Endettement",
        marker_color = "orange"
    )
)

In [105]:
df.transpose().plot(kind = "bar", y = ["Dette BOP","Dette EOP","Intérêts financiers","Remboursement","Tirage"]).update_layout(hovermode='x',barmode='group')

In [106]:
dd = df / df.loc["Coef. actualisation"]
dd

,-2,-1,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,30
Coef. actualisation,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
CAPEX,-661500.0,-630000.0,-600000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
OPEX,0.0,0.0,0.000000,-34285.714286,-32653.061224,-31098.153547,-29617.289093,-28206.941993,-26863.754279,-25584.527885,...,-1.292193e+04,-1.230660e+04,-1.172057e+04,-1.116244e+04,-1.063090e+04,-1.012467e+04,-9.642539e+03,-9.183371e+03,-8.746068e+03,-8.329588e+03
Prod. (MWh),NaN,NaN,NaN,2095.238095,1995.464853,1900.442717,1809.945445,1723.757566,1641.673873,1563.498926,...,7.896732e+02,7.520697e+02,7.162569e+02,6.821494e+02,6.496661e+02,6.187296e+02,5.892663e+02,5.612060e+02,5.344819e+02,5.090304e+02
Revenus,0.0,0.0,0.000000,157142.857143,149659.863946,142533.203758,135745.908341,129281.817467,123125.540445,117262.419471,...,4.343203e+04,4.136383e+04,3.939413e+04,3.751822e+04,3.573164e+04,3.403013e+04,3.240965e+04,3.086633e+04,2.939650e+04,2.799667e+04
EBITDA,0.0,0.0,0.000000,122857.142857,117006.802721,111435.050211,106128.619248,101074.875474,96261.786166,91677.891587,...,3.051010e+04,2.905724e+04,2.767356e+04,2.635577e+04,2.510074e+04,2.390546e+04,2.276711e+04,2.168296e+04,2.065044e+04,1.966708e+04
Amortissements,0.0,0.0,0.000000,-114285.714286,-108843.537415,-103660.511824,-98724.296975,-94023.139976,-89545.847596,-85281.759616,...,-7.178847e+03,-6.836997e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
EBIT,0.0,0.0,0.000000,8571.428571,8163.265306,7774.538387,7404.322273,7051.735498,6715.938570,6396.131971,...,2.333125e+04,2.222024e+04,2.767356e+04,2.635577e+04,2.510074e+04,2.390546e+04,2.276711e+04,2.168296e+04,2.065044e+04,1.966708e+04
Dette BOP,441000.0,385000.0,333333.333333,285714.285714,241874.527589,201562.106324,164540.494958,208940.311058,166654.771916,127922.639423,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Tirage,441000.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,78352.616647,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [107]:
fig = dd.transpose().plot(kind = "bar", y = ["CAPEX","OPEX","Revenus","Impôts (IS)"]).update_layout(hovermode='x')
fig.add_trace(
    go.Scatter(
        x = dd.columns,
        y = dd.loc["Cash Flow"],
        name = "Cash Flow",
        marker_color = "black"
    )
)

In [144]:
def groupement_année(k):
    if k<=0:
        return "Années négatives"
    else:
        return f"Années {((k-1)//5)*5+1}-{((k-1)//5+1)*5}"
de = pd.melt(dd.loc[["OPEX","CAPEX","Revenus","Impôts (IS)","Tirage","Remboursement","Intérêts financiers"]].reset_index(),id_vars="index").rename({"variable":"Année projet","index":"Type de flux"},axis=1)
de["value"] = de["value"].abs().round()
de["Période projet"] = de["Année projet"].apply(groupement_année)
de

,Type de flux,Année projet,value,Période projet
0,OPEX,-2,0.0,Années négatives
1,CAPEX,-2,661500.0,Années négatives
2,Revenus,-2,0.0,Années négatives
3,Impôts (IS),-2,0.0,Années négatives
4,Tirage,-2,441000.0,Années négatives
...,...,...,...,...
226,Revenus,30,27997.0,Années 26-30
227,Impôts (IS),30,5900.0,Années 26-30
228,Tirage,30,0.0,Années 26-30
229,Remboursement,30,0.0,Années 26-30


In [145]:
fig = px.treemap(
    de, 
    path = [px.Constant("Ensemble du projet"), "Type de flux", "Période projet", "Année projet"], 
    values="value",
    maxdepth=3
)
fig.data[0].textinfo = 'label+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(title = "Valeur actualisé des flux du projet")
fig.show()

In [10]:
npf.irr([30,-10*1.1,-10*1.1**2,-10*1.1**3])*100

9.999999999999986

In [152]:
aa = pd.DataFrame(dd.loc[["CAPEX","OPEX","Revenus","Impôts (IS)","Tirage","Remboursement","Intérêts financiers"]].sum(1),columns=["Initial"])
aa["Positif"] = dd.sum(1).apply(lambda x : 0 if x < 0 else x)
aa["Négatif"] = -dd.sum(1).apply(lambda x : 0 if x > 0 else x)
fig = go.Figure()
list_to_show = ["CAPEX","OPEX","Revenus","Impôts (IS)","Tirage","Remboursement","Intérêts financiers"]
for flux in list_to_show:
    fig.add_trace(
        go.Bar(
        x = aa[["Positif","Négatif"]].columns,
        y = aa[["Positif","Négatif"]].loc[flux],
        name = flux
        )
    )

    fig.update_layout(
    barmode = "relative",
    title = "Valeur actualisée net des flux financiers du projet",
    xaxis_title = "Valeur (€)"
    )

fig.show()

In [151]:
aa

,Initial,Positif,Négatif
CAPEX,-1.960271e+06,0.000000e+00,1.960271e+06
OPEX,-5.534082e+05,0.000000e+00,5.534082e+05
Revenus,2.316772e+06,2.316772e+06,-0.000000e+00
Impôts (IS),-2.298750e+05,0.000000e+00,2.298750e+05
Tirage,5.193526e+05,5.193526e+05,-0.000000e+00
Remboursement,-4.055376e+05,0.000000e+00,4.055376e+05
Intérêts financiers,-5.591305e+04,0.000000e+00,5.591305e+04


In [111]:
import numpy_financial as npf

In [112]:
npf.npv(5/100,df.loc["Cash Flow"])*(1+5/100)**2

36656.59660367117

In [113]:
npf.irr(df.loc["Cash Flow"])*100

5.258755981310093

In [1]:
import streamlit_static_pages as static

ModuleNotFoundError: No module named 'streamlit_static_pages'